In [1]:
import os
import spacy
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

In [2]:
from s3 import S3FileManager

AWS_BUCKET = os.getenv("AWS_BUCKET_NAME")
S3_PREFIX = "documents/"

In [3]:
AWS_BUCKET = os.getenv("AWS_BUCKET_NAME")
S3_PREFIX = "documents/"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
EMBEDDING_MODEL = "text-embedding-3-small"

In [4]:
doc1 = """
output values. These are concatenated and once again projected, resulting in the final values, as depicted in Figure 2.

Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this.

<!-- formula-not-decoded -->

<!-- formula-not-decoded -->

Where the projections are parameter matrices W Q i ∈ R d model × d k , W K i ∈ R d model × d k , W V i ∈ R d model × d v and W O ∈ R hd v × d model .

In this work we employ h = 8 parallel attention layers, or heads. For each of these we use d k = d v = d model /h = 64 . Due to the reduced dimension of each head, the total computational cost is similar to that of single-head attention with full dimensionality.

## 3.2.3 Applications of Attention in our Model

The Transformer uses multi-head attention in three different ways:

- · In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9].
- · The encoder contains self-attention layers. In a self-attention layer all of the keys, values and queries come from the same place, in this case, the output of the previous layer in the encoder. Each position in the encoder can attend to all positions in the previous layer of the encoder.
- · Similarly, self-attention layers in the decoder allow each position in the decoder to attend to all positions in the decoder up to and including that position. We need to prevent leftward information flow in the decoder to preserve the auto-regressive property. We implement this inside of scaled dot-product attention by masking out (setting to -∞ ) all values in the input of the softmax which correspond to illegal connections. See Figure 2.

## 3.3 Position-wise Feed-Forward Networks

In addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully connected feed-forward network, which is applied to each position separately and identically. This consists of two linear transformations with a ReLU activation in between.

<!-- formula-not-decoded -->

While the linear transformations are the same across different positions, they use different parameters from layer to layer. Another way of describing this is as two convolutions with kernel size 1. The dimensionality of input and output is d model = 512 , and the inner-layer has dimensionality d ff = 2048 .

## 3.4 Embeddings and Softmax

Similarly to other sequence transduction models, we use learned embeddings to convert the input tokens and output tokens to vectors of dimension d model. We also use the usual learned linear transformation and softmax function to convert the decoder output to predicted next-token probabilities. In our model, we share the same weight matrix between the two embedding layers and the pre-softmax linear transformation, similar to [30]. In the embedding layers, we multiply those weights by √ d model.
"""

In [5]:
documents = [doc1]

In [6]:
def semantic_chunking(text, max_sentences=5):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    
    chunks = []
    current_chunk = []
    for i, sent in enumerate(sentences):
        current_chunk.append(sent)
        if (i + 1) % max_sentences == 0:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks

In [7]:
semantic_chunks = semantic_chunking(doc1)
semantic_chunks[2]

'= 64 . Due to the reduced dimension of each head, the total computational cost is similar to that of single-head attention with full dimensionality.\n\n ## 3.2.3 Applications of Attention in our Model\n\nThe Transformer uses multi-head attention in three different ways:\n\n- · In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence.'

In [ ]:
def generate_embeddings(texts):
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=OPENAI_API_KEY,
                model_name="text-embedding-3-small"
            )
    return openai_ef(texts)

In [9]:
value = generate_embeddings(semantic_chunks)


In [10]:
value

[array([ 0.04584392,  0.02012629,  0.08212437, ..., -0.00191965,
         0.00995562,  0.00498413], shape=(1536,), dtype=float32),
 array([ 0.00951175,  0.01475875,  0.02481758, ...,  0.0153307 ,
        -0.00456315,  0.01047536], shape=(1536,), dtype=float32),
 array([ 0.02812611,  0.01595369,  0.06689579, ..., -0.00772593,
         0.04551528,  0.03916648], shape=(1536,), dtype=float32),
 array([ 0.06543998,  0.00956596, -0.00124241, ..., -0.0101656 ,
         0.03213166,  0.00421162], shape=(1536,), dtype=float32),
 array([ 0.03179673, -0.03211245,  0.03272132, ...,  0.00229032,
        -0.00096757,  0.04713133], shape=(1536,), dtype=float32),
 array([ 0.01797471, -0.02583332,  0.00384412, ..., -0.01113836,
         0.00710789,  0.0187414 ], shape=(1536,), dtype=float32),
 array([ 0.02663533, -0.00029789,  0.04269256, ..., -0.01071228,
        -0.02023927,  0.01938944], shape=(1536,), dtype=float32)]

## Test

In [11]:
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection("nvidia_document_chunks") # Create the collection once

for doc in documents:
    semantic_chunks = semantic_chunking(doc)
    embeddings = generate_embeddings(semantic_chunks)
    ids = [f"id_{i}" for i in range(len(semantic_chunks))]
    collection.add(
        documents=semantic_chunks,
        ids=ids,
        embeddings=embeddings,
    )

In [28]:
from pprint import pprint

query = ["multi-head attention in transformers"]
query_embeddings = generate_embeddings(query)
results = collection.query(
    query_embeddings=query_embeddings,
    n_results=2, # how many results to return
    where_document={'$contains': 'encoder'}
)

pprint(results)

{'data': None,
 'distances': [[0.5198990702629089, 0.9386508464813232]],
 'documents': [['= 64 . Due to the reduced dimension of each head, the total '
                'computational cost is similar to that of single-head '
                'attention with full dimensionality.\n'
                '\n'
                ' ## 3.2.3 Applications of Attention in our Model\n'
                '\n'
                'The Transformer uses multi-head attention in three different '
                'ways:\n'
                '\n'
                '- · In "encoder-decoder attention" layers, the queries come '
                'from the previous decoder layer, and the memory keys and '
                'values come from the output of the encoder. This allows every '
                'position in the decoder to attend over all positions in the '
                'input sequence.',
                'This mimics the typical encoder-decoder attention mechanisms '
                'in sequence-to-sequence models such

In [18]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=OPENAI_API_KEY,
                model_name="text-embedding-3-small"
            )
test_out = openai_ef(["multi-head attention in transformers"])

In [20]:
print(type(test_out))
test_out

<class 'list'>


[array([ 0.01688255,  0.0042638 ,  0.04444465, ..., -0.00435012,
         0.05998301,  0.01123448], shape=(1536,), dtype=float32)]

In [13]:
pprint(query_embeddings)

[array([ 0.0238633 ,  0.0172598 , -0.01232843, ..., -0.02780273,
        0.00091489,  0.00114959], shape=(1536,), dtype=float32),
 array([ 0.01288096,  0.01655534,  0.03272534, ..., -0.03101889,
       -0.00297081,  0.00994971], shape=(1536,), dtype=float32),
 array([-0.00323648, -0.01886252, -0.02487887, ..., -0.0026264 ,
        0.0031348 ,  0.01547831], shape=(1536,), dtype=float32),
 array([-0.01111129,  0.01217269, -0.00390046, ...,  0.00372232,
        0.01585419,  0.01134139], shape=(1536,), dtype=float32),
 array([ 0.01967479,  0.01060167, -0.00828781, ..., -0.02843939,
       -0.01295759,  0.01723472], shape=(1536,), dtype=float32),
 array([ 0.02252617,  0.01233576,  0.01070343, ..., -0.00250485,
       -0.00036557,  0.03681295], shape=(1536,), dtype=float32),
 array([ 0.01368417,  0.00645424,  0.01940664, ..., -0.02777813,
       -0.02600724,  0.01240357], shape=(1536,), dtype=float32),
 array([ 0.0197254 , -0.00858011, -0.01231732, ..., -0.01707176,
        0.01143277,  0.04

## Test ENd

In [ ]:
def store_in_chroma(chunks, metadata):
    """Store chunks in ChromaDB with metadata"""
    chroma_client = chromadb.Client()
    collection = chroma_client.get_or_create_collection("nvidia_document_chunks")
    texts = [chunk['text'] for chunk in chunks]
    embeddings = generate_embeddings(texts)
    metadatas = [chunk['metadata'] for chunk in chunks]
    ids = [f"{metadata['source']}_{i}" for i in range(len(chunks))]
    
    collection.add(
        ids=ids,
        embeddings=embeddings,
        metadatas=metadatas,
        documents=texts
    )

In [ ]:
documents = [doc1]
for doc in documents:
        # Base metadata
        base_metadata = {
            "source": doc['key'],
            "year": doc['year'],
            "quarter": doc['quarter'],
            "file_size": len(doc['content'])
        }
        
        # Process with both chunking strategies
        all_chunks = []
        
        # Strategy 1: Semantic chunking
        semantic_chunks = semantic_chunking(doc['content'])
        for i, chunk in enumerate(semantic_chunks):
            all_chunks.append({
                "text": chunk,
                "metadata": {
                    **base_metadata,
                    "chunk_strategy": "semantic",
                    "chunk_num": i+1
                }
            })

        